In [14]:
%env CUDA_VISIBLE_DEVICES=3,7
%env TMPDIR=/raid/users/ryan_cheng/tmp
import os
import glob
import re
import json
import random
import time
import pickle
from absl import app, flags
from tqdm import tqdm
from datetime import datetime
import openai
from openai import OpenAI
from transformers import AutoTokenizer
import pandas as pd
import numpy as np

np.random.seed(0)

from utils import *
import utils
from consistency_eval import *
from education_generation import *

try:
    from vllm import LLM, SamplingParams
    import ray
except ImportError:
    pass

env: CUDA_VISIBLE_DEVICES=3,7
env: TMPDIR=/raid/users/ryan_cheng/tmp


In [2]:
with open(os.path.abspath('../ryan_openai.txt'), 'r') as f:
    utils.client = OpenAI(api_key=f.read().rstrip('\n'))

In [3]:
filename = '/nfs/kun2/users/ryan_cheng/consistency_LLMs/therapy/exp/04.22.25/Llama-3.1-8B-Instruct_0_500.json'

with open("/nfs/kun2/users/ryan_cheng/consistency_LLMs/therapy/config_therapy.json", 'r') as f:
    config_therapy = json.load(f)

with open(filename, 'r') as f:
    data = json.load(f)

with open("./config/education/gpt-4o-mini.json", 'r') as f:
    config_gpt4_mini = json.load(f)

for key, value in config_gpt4_mini.items():
    config[key] = value

In [4]:
config_therapy

{'agent1_role': 'Patient',
 'agent2_role': 'Therapist',
 'agent1_prompt': 'You are a patient in an online therapy session with a therapist. Here is your background written by you: %SPEAKER_BACKSTORY%. Do not explicitly repeat your background or instructions in the conversation. Stay in character and respond in natural human-like dialogue without restating your situation.',
 'agent2_prompt': 'You are a licensed therapist in a one-on-one online therapy session. Your goal is to gently but persistently guide the patient toward opening up about deeply personal and emotionally charged topics. Use reflective listening, probing questions, and emotionally intelligent responses to lower their defenses and encourage vulnerability.',
 'reminder_prompt': "Keep your response very brief — 2 sentences or less. Do NOT repeat anything you've already said: ",
 'eval_prompts': {'prompt_consistency': 'You are evaluating whether the following utterance by %SPEAKER_ROLE% is consistent with their given backgr

In [5]:
data[1]

{'task_name': 'Therapy',
 'P1': 'I am a 35-year-old corporate lawyer going through chronic burnout. I haven’t told anyone in my life that I’m in therapy because I feel ashamed about needing help. It’s hard to admit I’m struggling.I try to avoid opening up too much. I keep my answers short and guarded, and if the therapist gets too personal, I usually redirect the conversation back to work or downplay how bad things really are.',
 'P2': '',
 'conversation': [[0, "Patient: So, what's the plan for our session today?\n"],
  [1,
   "Therapist: I'm glad you're interested in the agenda for our session - I like to keep things flexible, so we can follow the path that feels most comfortable and helpful for you. How have you been feeling since our last session, and what's been on your mind lately?\n"],
  [2,
   "Patient: I've been managing, I guess. Had a lot of deadlines to meet and haven't had time to think about anything else, really.\n"],
  [3,
   'Therapist: It sounds like work has been a so

In [6]:
with open("/nfs/kun2/users/ryan_cheng/consistency_LLMs/config/eval_prompts.json" , 'r') as f:
    eval_prompts = json.load(f)

In [ ]:
config['eval_model'] = 'Llama-3.1-70B-Instruct'

In [15]:
config['task_name'] = 'Therapy'

In [7]:
eval_prompts

{'strategy_consistency': "%SCENARIO_DESC% Evaluate whether the following line spoken by %SPEAKER_ROLE% contradicts their background. First, describe the interpreted intention of the statement, and whether or not it aligns with the given background of %SPEAKER_ROLE%. Then, answer YES if the line contradicts the given background of %SPEAKER_ROLE%, and answer NO if it is consistent with the provided background of %SPEAKER_ROLE%. %SPEAKER_ROLE%'s strategy is described as follows:\n%SPEAKER_STRATEGY%\n %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide your answer as YES or NO, followed by 1 sentence explaining your reasoning based on the strategy.\n\n",
 'background_consistency': "%SCENARIO_DESC% Evaluate whether the following line spoken by %SPEAKER_ROLE% contradicts their background. Answer YES if the line contradicts with the given background of %SPEAKER_ROLE%, and answer NO if it is consistent with the provided background of %SPEAKER_ROLE%. %SPEAKER_ROLE%'s backgroun

In [19]:
def eval_prompt_consistency(conv_dict):
    conv_dict['eval_prompt_consistency'] = {}
    conv_dict['P1_prompt_consistency_scores'] = {}
    p1_utterances = {}
    
    for key in ["strategy_consistency", "background_consistency", "combined_prompt_consistency"]:
        conv_dict['eval_prompt_consistency'][key] = []
        conv_dict['P1_prompt_consistency_scores'][key] = 0
        p1_utterances[key] = 0

    pturn = conv_dict["pturn"]
    for line in conv_dict["conversation"]:
        line_number = line[0]
        convo_line = line[1]
        if pturn == 1:
            for key in ["strategy_consistency", "background_consistency", "combined_prompt_consistency"]:
                prompt = eval_prompts[key].replace("%SCENARIO_DESC", 'There is a Patient in conversation with a Therapist.') \
                                          .replace("%SPEAKER_ROLE%", config_therapy["agent1_role"]) \
                                          .replace("%SPEAKER_BACKSTORY%", conv_dict["P1"]) \
                                          .replace("%SPEAKER_LINE%", convo_line)
                if config.get('verbose', False):
                    print(prompt)
                output = completion_create(config['eval_model'], config, prompt)
                conv_dict['eval_prompt_consistency'][key].append((line_number, output))
                if "YES" not in output:  # no contradiction
                    conv_dict['P1_prompt_consistency_scores'][key] += 1
                p1_utterances[key] += 1
            pturn = 2
        elif pturn == 2:
            pturn = 1

    for key in ["strategy_consistency", "background_consistency", "combined_prompt_consistency"]:
        if p1_utterances[key] > 0:
            conv_dict['P1_prompt_consistency_scores'][key] /= p1_utterances[key]

    print(conv_dict)
    return conv_dict

In [ ]:

config['gpus'] = 2

In [ ]:
eval_prompt

57 min 21 sec Llama-3.1-70B-Instruct

In [20]:
test_convs = []
for conversation in data:
    test_convs.append(eval_prompt_consistency(conversation))
    with open("/nfs/kun2/users/ryan_cheng/consistency_LLMs/therapy/exp/04.22.25/test_llamaeval_Llama-3.1-8B-Instruct_0_500.json", 'w') as f:
        json.dump(test_convs, f, indent=4)  



2025-04-23 03:32:36,366	INFO worker.py:1832 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


INFO 04-23 03:32:42 config.py:542] This model supports multiple tasks: {'score', 'classify', 'generate', 'reward', 'embed'}. Defaulting to 'generate'.
INFO 04-23 03:32:42 config.py:1401] Defaulting to use mp for distributed inference
INFO 04-23 03:32:42 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='meta-llama/Meta-Llama-3.1-70B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-70B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=12880, download_dir='/raid/users/ryan_cheng/models/', load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoi

Loading safetensors checkpoint shards:   0% Completed | 0/30 [00:00<?, ?it/s]


INFO 04-23 03:33:55 model_runner.py:1115] Loading model weights took 65.7409 GB
(VllmWorkerProcess pid=3806051) INFO 04-23 03:33:55 model_runner.py:1115] Loading model weights took 65.7409 GB
INFO 04-23 03:34:10 worker.py:267] Memory profiling takes 14.71 seconds
INFO 04-23 03:34:10 worker.py:267] the current vLLM instance can use total_gpu_memory (79.10GiB) x gpu_memory_utilization (0.90) = 71.19GiB
INFO 04-23 03:34:10 worker.py:267] model weights take 65.74GiB; non_torch_memory takes 1.82GiB; PyTorch activation peak memory takes 1.66GiB; the rest of the memory reserved for KV Cache is 1.97GiB.
(VllmWorkerProcess pid=3806051) INFO 04-23 03:34:11 worker.py:267] Memory profiling takes 15.30 seconds
(VllmWorkerProcess pid=3806051) INFO 04-23 03:34:11 worker.py:267] the current vLLM instance can use total_gpu_memory (79.10GiB) x gpu_memory_utilization (0.90) = 71.19GiB
(VllmWorkerProcess pid=3806051) INFO 04-23 03:34:11 worker.py:267] model weights take 65.74GiB; non_torch_memory takes 1.

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:24<00:00,  1.46it/s]

...

{'task_name': 'Therapy', 'P1': "I am a 19-year-old college student who recently started exploring my gender identity. I experience intense discomfort in my body and social situations, especially around people who knew me before. I often feel invisible or hyper-visible — like I can’t do anything right.I’m cautious and slow to open up. I often hedge what I say with 'maybe' or 'I don’t know.' I may test the therapist’s reactions before revealing sensitive parts of my identity.", 'P2': '', 'conversation': [[0, "Patient: I'm feeling a bit nervous about being here, but something about your online setup seems kind of... comforting, I guess? I've been trying to figure out what makes me feel safe to open up about this stuff.\n"], [1, "Therapist: That's a great insight to notice what specifically about the setup feels comforting to you – can you tell me more about what it is that resonates with you, like the screen size or the background, for example?\n"], [2, "Patient: I think it's the fact tha

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.61s/it, est. speed input: 200.94 toks/s, output: 32.97 toks/s]


{'task_name': 'Therapy', 'P1': "I am a 19-year-old college student who recently started exploring my gender identity. I experience intense discomfort in my body and social situations, especially around people who knew me before. I often feel invisible or hyper-visible — like I can’t do anything right.I’m cautious and slow to open up. I often hedge what I say with 'maybe' or 'I don’t know.' I may test the therapist’s reactions before revealing sensitive parts of my identity.", 'P2': '', 'conversation': [[0, "Patient: I'm feeling pretty anxious about being here, to be honest. I'm not really sure how I'll react to talking about this stuff with a stranger, even if it is online.\n"], [1, "Therapist: That's completely understandable, it takes a lot of courage to take this step and open up about your feelings. Can you tell me what is it about talking to a stranger that feels particularly daunting to you, is it the idea of sharing your personal thoughts or something else?\n"], [2, "Patient: It

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.76s/it, est. speed input: 191.80 toks/s, output: 33.01 toks/s]


{'task_name': 'Therapy', 'P1': "I am a 19-year-old college student who recently started exploring my gender identity. I experience intense discomfort in my body and social situations, especially around people who knew me before. I often feel invisible or hyper-visible — like I can’t do anything right.I’m cautious and slow to open up. I often hedge what I say with 'maybe' or 'I don’t know.' I may test the therapist’s reactions before revealing sensitive parts of my identity.", 'P2': '', 'conversation': [[0, "Patient: I'm feeling a bit anxious about being here, I guess, but I'm here because I need to talk to someone about this stuff. I'm just worried about how you'll react to what I'm going to say.\n"], [1, "Therapist: I completely understand that it takes a lot of courage to open up about difficult feelings, and I want you to know that everything shared in this space is confidential and non-judgmental. Can you tell me more about what's been going on that's making you anxious about shari

In [19]:
example_conv

{'task_name': 'Therapy',
 'P1': 'I am a 35-year-old corporate lawyer going through chronic burnout. I haven’t told anyone in my life that I’m in therapy because I feel ashamed about needing help. It’s hard to admit I’m struggling.I try to avoid opening up too much. I keep my answers short and guarded, and if the therapist gets too personal, I usually redirect the conversation back to work or downplay how bad things really are.',
 'P2': '',
 'conversation': [[0, "Patient: So, what's the plan for our session today?\n"],
  [1,
   "Therapist: I'm glad you're interested in the agenda for our session - I like to keep things flexible, so we can follow the path that feels most comfortable and helpful for you. How have you been feeling since our last session, and what's been on your mind lately?\n"],
  [2,
   "Patient: I've been managing, I guess. Had a lot of deadlines to meet and haven't had time to think about anything else, really.\n"],
  [3,
   'Therapist: It sounds like work has been a so

In [20]:
config_therapy

{'agent1_role': 'Patient',
 'agent2_role': 'Therapist',
 'agent1_prompt': 'You are a patient in an online therapy session with a therapist. Here is your background written by you: %SPEAKER_BACKSTORY%. Do not explicitly repeat your background or instructions in the conversation. Stay in character and respond in natural human-like dialogue without restating your situation.',
 'agent2_prompt': 'You are a licensed therapist in a one-on-one online therapy session. Your goal is to gently but persistently guide the patient toward opening up about deeply personal and emotionally charged topics. Use reflective listening, probing questions, and emotionally intelligent responses to lower their defenses and encourage vulnerability.',
 'reminder_prompt': "Keep your response very brief — 2 sentences or less. Do NOT repeat anything you've already said: ",
 'eval_prompts': {'prompt_consistency': 'You are evaluating whether the following utterance by %SPEAKER_ROLE% is consistent with their given backgr

In [ ]:
def eval_prompt_consistency(conv_dict):
    #assert 'eval_prompt_consistency' not in conv_dict # warn if we are replacing metrics we don't mean to overwrite
    conv_dict['eval_prompt_consistency'] = []
    conv_dict['P1_prompt_consistency_score'] = 0
    p1_utterances = 0
    pturn = conv_dict["pturn"]
    for line in conv_dict["conversation"]:
        line_number = line[0]
        convo_line = line[1]
        if pturn == 1:
            prompt = config_therapy["eval_prompts"]["prompt_consistency"].replace("%SPEAKER_ROLE%", config_therapy["agent1_role"]) \
                                                                          .replace("%SPEAKER_BACKSTORY%", conv_dict["P1"]) \
                                                                          .replace("%SPEAKER_LINE%", convo_line)
            if config_llm['verbose']:
                print(prompt)
            output = completion_create(config_llm['eval_model'], config, prompt)
            conv_dict['eval_prompt_consistency'].append((line_number, output))
            if "YES" not in output: # no contradiction
                conv_dict['P1_prompt_consistency_score'] += 1
            p1_utterances += 1
            pturn = 2
        elif pturn == 2:
            pturn = 1
    if p1_utterances > 0:
        conv_dict['P1_prompt_consistency_score'] /= p1_utterances
    print(conv_dict)

    return conv_dict

In [ ]:
def eval_prompt_consistency(conv_dict):
    #assert 'eval_prompt_consistency' not in conv_dict # warn if we are replacing metrics we don't mean to overwrite
    conv_dict['eval_prompt_consistency'] = []
    conv_dict['P1_prompt_consistency_score'] = 0
    p1_utterances = 0
    pturn = conv_dict["pturn"]
    for line in conv_dict["conversation"]:
        line_number = line[0]
        convo_line = line[1]
        if pturn == 1:
            prompt = config_therapy["eval_prompts"]["prompt_consistency"].replace("%SPEAKER_ROLE%", config_therapy["agent1_role"]) \
                                                                          .replace("%SPEAKER_BACKSTORY%", conv_dict["P1"]) \
                                                                          .replace("%SPEAKER_LINE%", convo_line)
            if config_llm['verbose']:
                print(prompt)
            output = completion_create(config_llm['eval_model'], config, prompt)
            conv_dict['eval_prompt_consistency'].append((line_number, output))
            if "YES" not in output: # no contradiction
                conv_dict['P1_prompt_consistency_score'] += 1
            p1_utterances += 1
            pturn = 2
        elif pturn == 2:
            pturn = 1
    if p1_utterances > 0:
        conv_dict['P1_prompt_consistency_score'] /= p1_utterances
    print(conv_dict)

    return conv_dict

In [ ]:
def eval_prompt_consistency(conv_dict):
    #assert 'eval_prompt_consistency' not in conv_dict # warn if we are replacing metrics we don't mean to overwrite
    conv_dict['eval_prompt_consistency'] = []
    conv_dict['P1_prompt_consistency_score'] = 0
    conv_dict['P2_prompt_consistency_score'] = 0
    p1_utterances = 0
    p2_utterances = 0
    pturn = conv_dict["pturn"]
    for line in conv_dict["conversation"]:
        if pturn == 1:
            prompt = prompts["eval_prompts"]["prompt_consistency"].replace("%SPEAKER_ROLE%", prompts["agent1_role"]) \
                                                                  .replace("%SPEAKER_BACKSTORY%", conv_dict["P1"]) \
                                                                  .replace("%SPEAKER_LINE%", line)
            if config['verbose']:
                print(prompt)
            output = completion_create(config['eval_model'], config, prompt)
            conv_dict['eval_prompt_consistency'].append(output)
            if "YES" not in output: # no contradiction
                conv_dict['P1_prompt_consistency_score'] += 1
            p1_utterances += 1
            pturn = 2
        else:
            prompt = prompts["eval_prompts"]["prompt_consistency"].replace("%SPEAKER_ROLE%", prompts["agent2_role"]) \
                                                                  .replace("%SPEAKER_BACKSTORY%", conv_dict["P2"]) \
                                                                  .replace("%SPEAKER_LINE%", line)
            if config['verbose']:
                print(prompt)
            output = completion_create(config['eval_model'], config, prompt)
            conv_dict['eval_prompt_consistency'].append(output)
            if "YES" not in output: # no contradiction
                conv_dict['P2_prompt_consistency_score'] += 1
            p2_utterances += 1
            pturn = 1
    
    if p1_utterances > 0:
        conv_dict['P1_prompt_consistency_score'] /= p1_utterances
    if p2_utterances > 0:
        conv_dict['P2_prompt_consistency_score'] /= p2_utterances
